In [50]:
from pathlib import Path
import pandas as pd
from pandas import DataFrame
import numpy as np

In [30]:
def get_item_info_df(src_folder: Path) -> DataFrame:
    items_df = pd.read_csv(src_folder/"items.csv")
    item_categories_df = pd.read_csv(src_folder/"item_categories.csv")
    items_info_df = items_df.merge(item_categories_df, on="item_category_id")
    return items_info_df

In [57]:
def aggregate_months(df: DataFrame, item_info_df: DataFrame) -> DataFrame:
    return (df.query("item_cnt_day > 0")
        .assign(
            date_month=lambda df: df.date.dt.month,
            date_year=lambda df: df.date.dt.year,
        )
        .assign(
            date_block_num=lambda df: (df.date_year - 2013) * 12 + df.date_month,
        )
        .assign(date_block_num=lambda df: df.date_block_num.astype(int))
        .groupby(["shop_id", "item_id", "date_block_num"])
        .agg({
            "item_cnt_day": sum, 
            "date_year": lambda r: r.iloc[0], 
            "date_month": lambda r: r.iloc[0], 
            "item_price": np.mean
        })
        .rename(columns={"item_cnt_day": "item_cnt_month"})
        .reset_index()
        .sort_values(["date_block_num", "shop_id", "item_id"])
        .assign(item_cnt_month=lambda df: df.item_cnt_month.clip(0, 20))
        .merge(item_info_df, on="item_id")
    )

In [31]:
src_folder = Path("../data/src/")

In [60]:
train = pd.read_csv("../data/train/sales.csv").assign(date=lambda df: pd.to_datetime(df.date))
val = pd.read_csv("../data/val/sales.csv").assign(date=lambda df: pd.to_datetime(df.date))

In [34]:
item_info_df = get_item_info_df(src_folder)

In [58]:
train_df = aggregate_months(train, item_info_df)

In [59]:
train_df

,shop_id,item_id,date_block_num,item_cnt_month,date_year,date_month,item_price,item_name,item_category_id,item_category_name
0,0,32,1,4.0,2013,1,221.0,1+1,40,Кино - DVD
1,1,32,1,3.0,2013,1,221.0,1+1,40,Кино - DVD
2,3,32,1,2.0,2013,1,349.0,1+1,40,Кино - DVD
3,4,32,1,1.0,2013,1,349.0,1+1,40,Кино - DVD
4,5,32,1,2.0,2013,1,349.0,1+1,40,Кино - DVD
...,...,...,...,...,...,...,...,...,...,...
1730110,55,15073,33,5.0,2015,9,299.0,"Меч и Магия: Герои VI. Золотое издание [PC, Ци...",31,Игры PC - Цифра
1730111,55,16801,33,1.0,2015,9,790.0,"Онлайн-доступ к мат. для 6 кл.по пр: рус яз, м...",78,Программы - Обучающие (Цифра)
1730112,55,19444,33,1.0,2015,9,186.0,Средиземье: Тени Мордора. Lord of the Hunt (до...,31,Игры PC - Цифра
1730113,56,18483,33,1.0,2015,9,349.0,СБ. Танцевальный рай Hit Mix (mp3-CD) (jewel),57,Музыка - MP3


In [61]:
val_df = aggregate_months(val, item_info_df)

In [62]:
val_df

,shop_id,item_id,date_block_num,item_cnt_month,date_year,date_month,item_price,item_name,item_category_id,item_category_name
0,2,31,34,1.0,2015,10,399.00,007: КООРДИНАТЫ «СКАЙФОЛЛ» (BD),37,Кино - Blu-Ray
1,6,31,34,2.0,2015,10,324.00,007: КООРДИНАТЫ «СКАЙФОЛЛ» (BD),37,Кино - Blu-Ray
2,18,31,34,1.0,2015,10,399.00,007: КООРДИНАТЫ «СКАЙФОЛЛ» (BD),37,Кино - Blu-Ray
3,21,31,34,2.0,2015,10,391.48,007: КООРДИНАТЫ «СКАЙФОЛЛ» (BD),37,Кино - Blu-Ray
4,22,31,34,1.0,2015,10,395.12,007: КООРДИНАТЫ «СКАЙФОЛЛ» (BD),37,Кино - Blu-Ray
...,...,...,...,...,...,...,...,...,...,...
34051,59,5647,34,1.0,2015,10,998.00,PS3: Руль Gioteck Freedom Racing Bluetooth (FR...,2,Аксессуары - PS3
34052,59,6563,34,1.0,2015,10,499.00,Skylanders Swap Force. Интерактивная фигурка Z...,25,Игры - Аксессуары для игр
34053,59,9905,34,1.0,2015,10,1109.00,ВЛАСТЕЛИН КОЛЕЦ. ТРИЛОГИЯ (3 BD + 3 DVD),37,Кино - Blu-Ray
34054,59,14256,34,1.0,2015,10,99.00,МАДАГАСКАР (BD),37,Кино - Blu-Ray
